In [128]:
import pandas as pd
from uber_rides.session import Session
from uber_rides.client import UberRidesClient
import random
from random import randint
from gmaps import Geocoding
import uber_rides.errors as uber_error

session = Session(server_token='gQqt-C_l3O2CSZ0Y2TNjIRJQmSmowvcJPMAOBLoB')
client = UberRidesClient(session)

In [142]:
# when looking for a cab pass seatcount as 1
# when booking a cab pass the exact seat count required
def look_uber(source, seatcount=2, destination=None):
    api = Geocoding()
    try:
        source_loc = api.geocode(source)
        source_lat_long = source_loc[0]['geometry']['location']
        start_lat=source_lat_long['lat']
        start_lng=source_lat_long['lng']
    except:
        return ('Enter Valid Source.')
    
    
    if destination : 
        try:
            destination_loc = api.geocode(destination)
            destination_lat_long = destination_loc[0]['geometry']['location']
            end_lat=destination_lat_long['lat']
            end_lng=destination_lat_long['lng']

        except:
            return ('Enter Valid destination.')
        
    else:
        end_lat=12.9173312 # latitude of Central Silk Board
        end_lng=77.6212483 # longitude of Central Silk Board
    try:
        response = client.get_price_estimates(
            start_latitude=start_lat,
            start_longitude=start_lng,
            end_latitude=end_lat,
            end_longitude=end_lng,
            seat_count=seatcount
            )    

        estimate = response.json 
        cab_details_df= uber_cab_details(estimate)
        cab_details_df.fillna(0, axis=1, inplace=True)
        return cab_details_df
    except uber_error.ClientError as e:
        return 'Distance between two points exceeds 100 miles'
    

In [130]:
def uber_cab_details(cab_dict):
    columns = ['Car type', 'Availability', 'Distance(Km)', 'Minimum Price', 'Maximum Price', 'ETA(min)' ]
    index=list(range(len(cab_dict['prices'])))
    df = pd.DataFrame(index=index, columns=columns)
    for i in index:
        df.loc[i,'Car type'] = estimate['prices'][i]['display_name']
        df.loc[i,'Availability'] = "Yes"
        df.loc[i,'Distance(Km)'] = estimate['prices'][i]['distance']
        df.loc[i,'Minimum Price'] = estimate['prices'][i]['low_estimate']
        df.loc[i,'Maximum Price'] = estimate['prices'][i]['high_estimate']
        df.loc[i,'ETA(min)'] = int(estimate['prices'][i]['duration']/100)
    return df      

In [131]:
def book_uber(source, destination, cabtype, noofseats):
    df = lookuber(noofseats,source, destination)
    driver_name=["Tharoon Veerasethu","Sharmaji","Chachaji","Chunnu","Munni","beta","daadaji","daadi","bhai","shinde","langoor"]
    for i in range(df.shape[0]):
        x=list(df.loc[i])
        if cabtype in x:
            for j in x:
                if j == cabtype and df.loc[i,'Availability']=='Yes':
                    SessInfo['bookingid']="UBER"+str(random_with_N_digits(6))
                    SessInfo['drivername']=random.choice(driver_name)
                    SessInfo['driverno']=random_with_N_digits(10)
                    SessInfo['carno']="KN-"+str(random_with_N_digits(2))+"-"+str(random_with_N_digits(4))
                    SessInfo['fare']=(df.loc[i,'Minimum Price']+df.loc[i,'Maximum Price'])/2
                    SessInfo['cabtype']=cabtype
                    SessInfo['noofseats']=noofseats
                    SessInfo['eta']=df.loc[i,'ETA(min)'] 
                    SessInfo['distance']=df.loc[i,'Distance(Km)']
                    break


In [132]:
def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)  

In [143]:
 look_uber('Electronic City, Bangalore', seatcount=2, destination='K R Puram')

,Car type,Availability,Distance(Km),Minimum Price,Maximum Price,ETA(min)
0,UberPOOL,Yes,12.15,265.0,326.0,25
1,UberGO,Yes,12.15,348.0,427.0,25
2,UberX,Yes,12.15,381.0,467.0,25
3,UberXL,Yes,12.15,573.0,701.0,25
